<a href="https://colab.research.google.com/github/mgerlach/advent_of_code/blob/main/2021/aoc2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) number of increments, window length 1 (part 1), window length 3 (part 2)

In [ ]:
wl = 1  # window length, use 3 for part 2
inc = 0 # increments

m = [int(line) for line in open('drive/MyDrive/AoC/2021/input01.txt')]

for i in range(0, len(m) - wl):
  if m[i] < m[i+wl]:
    inc += 1

inc

1722

2) horizontal/depth course

In [ ]:
prog = [line.split() for line in open('drive/MyDrive/AoC/2021/input02.txt')]  # 0=cmd 1=amount
h = 0
d = 0
for instr in prog:
  cmd = instr[0]
  amount = int(instr[1])
  if cmd == 'forward':
    h += amount
  elif cmd == 'up':
    d -= amount
  elif cmd == 'down':
    d += amount

h * d

1698735

In [ ]:
prog = [line.split() for line in open('drive/MyDrive/AoC/2021/input02.txt')]  # 0=cmd 1=amount
h = 0
d = 0
aim = 0 
for instr in prog:
  cmd = instr[0]
  amount = int(instr[1])
  if cmd == 'forward':
    h += amount
    d += aim * amount
  elif cmd == 'up':
    aim -= amount
  elif cmd == 'down':
    aim += amount

h * d


1594785890

3) Most / least common bits in binary numbers represented as equal length strings

In [ ]:
# count values per bit, keep either most common or least common value for each bit
from functools import reduce

ms = [line.strip() for line in open('drive/MyDrive/AoC/2021/input03.txt')]  # ms = measurements, binary 12 char strings
half = len(ms)/2

# count 1s in each bit position
initial_bit_count1s = [0] * len(ms[0])
count1s = reduce(lambda cnt1s, m: [cnt1s[i] + int(m[i]) for i, x in enumerate(cnt1s)], ms, initial_bit_count1s)

most_common_bitvals = "".join('1' if c > half else '0' for c in count1s)
least_common_bitvals = "".join('1' if c < half else '0' for c in count1s)  # could also invert most_common_bitvals

int(most_common_bitvals, 2) * int(least_common_bitvals, 2)

4138664

In [ ]:
# count values for first bit, keep lines with first bit either most or least common value,
# repeat with next bit on filtered list, repeat until only one list entry is left
measurements = [line.strip() for line in open('drive/MyDrive/AoC/2021/input03.txt')]  # binary 12 char strings
bits = len(measurements[0])

# find the measurement according to filtermode
# ms - measurements
# filtermode 1 - keep numbers with bit value = most common value of the bit (equally common - keep those with 1)
# filtermode 0 - keep numbers with bit value = least common value of the bit (equally common - keep those with 0)
def find(ms, filtermode):

  def filter(ms, bit):
    half = len(ms)/2
    # count 1s in bit position
    count1s = sum(int(m[bit]) for m in ms)
    # filter according to count1s and filtermode
    filterval = filtermode if count1s > half else (1-filtermode if count1s < half else filtermode)
    return [m for m in ms if int(m[bit]) == filterval]

  def iter(ms, bit):
    filtered = filter(ms, bit)
    return filtered[0] if len(filtered) == 1 else iter(filtered, bit+1)

  return iter(ms, 0)

int(find(measurements, 1), 2) * int(find(measurements, 0), 2)

4273224

4) Bingo (5x5 Boards)


In [ ]:
# Board structure
# 2-dimensional array of pairs (2-elem lists) with left(0) = number, right(1) = marker (1 = marked, 0 = not marked)
# The number/marker pairs are shared between all boards.

from functools import reduce

# Assume numbers drawn between 0 and 99 (incl.), initialize a marker object (a list length 2) for each number,
# containing the number itself (index 0, actually only needed for display as the index of the marker object 
# already represents the number) and a marker (index 1) of value 0 if the number has not been drawn, yet, and
# 1 if the number was drawn. These markers are shared between all the boards, such that setting the marker to 1
# in one of the numbers, the number will appear as marked on all boards containing it.

markers = [[n, 0] for n in range(0, 100)]
boards = []

for i, l in enumerate(open('drive/MyDrive/AoC/2021/input04.txt')):
  if i == 0:
    # The numbers that are drawn appear in the first line of the input
    numbers = [int(s.strip()) for s in l.split(',')]
  elif i % 6 == 1: 
    # A blank line announcing a new 5x5 board, lines 1, 7, 13, etc.
    boards.append([])
  else:
    # By using the numbers on the board def. as index into markers, the marker objects are shared between all the boards
    boards[int((i-1)/6)].append([markers[int(s.strip())] for s in l.split()])

# Some helpers

def mark(n):
  markers[n][1] = 1  # marks number on all boards

def rowbingo(board) -> bool:
  # bool(list) is True if non-empty, i. e. 5 marked numbers in at least 1 row
  return bool([row for row in board if sum(n[1] for n in row) == 5])  

def colbingo(board) -> bool:
  # The reduce counts marked numbers (row[i][1] == 1) in all columns (i=0..4)
  # true if 5 numbers are marked in any column
  return 5 in reduce(lambda counts, row: [c+row[i][1] for i, c in enumerate(counts)], board, [0, 0, 0, 0, 0])

def bingo(board) -> bool:
  return rowbingo(board) or colbingo(board)

def sum_unmarked(board) -> int:
  return sum(sum(n[0] for n in row if n[1] == 0) for row in board)

def p(board):
  for row in board:
    print(row)

# Part 1: First board to win
bingoboards = []
index = 0
while len(bingoboards) == 0:
  mark(numbers[index])
  bingoboards = list(filter(bingo, boards))  # [board for board in boards if bingo(board)]
  if bingoboards:
    print("First board")
    p(bingoboards[0])
    print(sum_unmarked(bingoboards[0]) * numbers[index])
  else:
    index += 1

# Part 2: Last board to win

# Now need to remove boards with bingo until only one left
# (first remove the one found above)
boards.remove(bingoboards[0])
while len(boards) > 1:
  # print(f"index: {index}, number of boards: {len(boards)}")
  mark(numbers[index])
  boards = [board for board in boards if not bingo(board)]  # list(filter(lambda board: not(bingo(board)), boards))
  index += 1

print("Last board")
# Go on until bingo
while index < len(numbers):
  mark(numbers[index])
  if (bingo(boards[0])):
    p(boards[0])
    print(sum_unmarked(boards[0]) * numbers[index])
    index = len(numbers)  # stop
  else:
    index += 1



First board
[[6, 0], [26, 0], [69, 1], [27, 1], [75, 0]]
[[61, 0], [33, 1], [88, 1], [38, 0], [20, 0]]
[[9, 1], [56, 0], [70, 0], [98, 1], [82, 0]]
[[80, 1], [76, 0], [55, 1], [66, 1], [29, 0]]
[[97, 1], [84, 1], [42, 1], [77, 1], [73, 1]]
41503
Last board
[[94, 1], [38, 1], [96, 1], [44, 1], [46, 0]]
[[10, 1], [14, 1], [75, 0], [97, 1], [76, 1]]
[[7, 1], [61, 1], [56, 1], [36, 1], [1, 0]]
[[81, 0], [67, 1], [49, 1], [78, 1], [86, 1]]
[[31, 1], [65, 1], [88, 1], [24, 0], [63, 1]]
3178


5) Overlapping lines

In [ ]:
# Part 1 only horizontal and vertical lines
# Part 2 also 45 degree diagonal lines
import re

grid = {}  # maps points (x, y) to count

# needed for part 2
def sgn(x): 
  return 1 if x > 0 else (-1 if x < 0 else 0)

def inc_point(x, y):
  if (x, y) not in grid:
    grid[(x, y)] = 1
  else:
    grid[(x, y)] += 1

def add_line(x1, y1, x2, y2):
  if x1==x2:
    for y in range(min(y1,y2), max(y1,y2)+1):
      inc_point(x1, y)
  elif y1==y2:
    for x in range(min(x1,x2), max(x1,x2)+1):
      inc_point(x, y1)
  # Part 2 - could also use this as single option without all the ifs (trusting the input) as it would cover horizontal and vertical lines, too
  elif abs(x2-x1)==abs(y2-y1):
    dx = sgn(x2-x1)
    dy = sgn(y2-y1)
    for i in range(0, abs(x2-x1)+1):
      inc_point(x1 + i * dx, y1 + i * dy)

for l in open('drive/MyDrive/AoC/2021/input05.txt'):
  m = re.fullmatch(r"(\d+),(\d+) -> (\d+),(\d+)\n", l)
  add_line(*[int(m.group(i)) for i in [1, 2, 3, 4]])

print(f"number of points where lines overlap: {sum(1 if grid[k] > 1 else 0 for k in grid.keys())}")

number of points where lines overlap: 22083


6) Population growth

In [ ]:
# Part 1 naive brute force
population = [int(timer_str) for timer_str in open('drive/MyDrive/AoC/2021/input06.txt').readline().split(',')]
iterations = 80
for i in range(0, iterations):
  # print(f"{i}: {len(population)}")
  new_timers = [8] * sum(1 if timer == 0 else 0 for timer in population)
  population = [6 if timer == 0 else timer-1 for timer in population]
  population += new_timers

print(f"{iterations}: {len(population)}")


80: 380612


In [ ]:
# Part 2 a bit more intelligent
pop_raw = [int(timer_str) for timer_str in open('drive/MyDrive/AoC/2021/input06.txt').readline().split(',')]
pop_counts = [0,0,0,0,0,0,0,0,0]  # counts for timer values 0-8
for timer in pop_raw:
  pop_counts[timer] += 1
iterations = 256
for i in range(0, iterations):
  # print(f"{i}: {pop_counts} ({sum(pop_counts)})")
  new_counts = pop_counts[1:]  # decrease all from 1-8
  new_counts[6] += pop_counts[0]  # 0s become 6s
  pop_counts = new_counts + [pop_counts[0]]  # 0s produce new 8s

print(f"{iterations}: {pop_counts} ({sum(pop_counts)})")


256: [145229151101, 175734577513, 183417971628, 196504736072, 231271638229, 222902115155, 282121656192, 119339467551, 153645343459] (1710166656900)


7) Find min diff

In [ ]:
positions = [int(s) for s in open('drive/MyDrive/AoC/2021/input07.txt').readline().split(',')]
# input = [16,1,2,0,4,2,7,1,2,14]
print(f"estimation(part 1): {sum(positions)/len(positions)/2}")

def consumption1(target, current):
  return sum(abs(target - curr) for curr in current)

def consumption2(target, current):
  return sum(int(abs(target-curr)*(abs(target-curr)+1)/2) for curr in current)

print("part 1:", min([(i, consumption1(i, positions)) for i in range(0, len(positions))], key=lambda t: t[1]))
print("part 2:", min([(i, consumption2(i, positions)) for i in range(0, len(positions))], key=lambda t: t[1]))


estimation(part 1): 243.2525
part 1: (347, 347449)
part 2: (486, 98039527)


8) 4-Digit display wiring

In [ ]:
patterns_and_output = [{'patterns': l[0].split(), 'output': l[1].split()} for l in [line.split('|') for line in open('drive/MyDrive/AoC/2021/input08.txt')]]

# segments for digits 0-9
segments = [ 'abcefg', 'cf', 'acdeg', 'acdfg', 'bcdf', 'abdfg', 'abdefg', 'acf', 'abcdefg', 'abcdfg' ]

# 1 unique digits in output

# segment counts by digit
sc = dict((i, len(s)) for i, s in enumerate(segments))

print("part 1:", sum(1 if len(o) in [sc[1], sc[4], sc[7], sc[8]] else 0 for po in patterns_and_output for o in po['output']))

# 2 find wiring from patterns, then translate/sort output to find the digits

# Segment letters for each digit - this is what the output must be translated to!
# 1:     c     f
# 7: a   c     f    
# 4:   b c d   f     
# 5: a b   d   f g
# 2: a   c d e   g
# 3: a   c d   f g
# 9: a b c d   f g
# 0: a b c   e f g 
# 6: a b   d e f g
# 8: a b c d e f g 

from functools import reduce

# output: map letter in input pattern -> segments (a-g)
def decode(patterns) -> dict:

  # sort patterns by number of segments used for the digit,
  # there are always 1 of length 2 (representing "1"), 1/3 ("7"), 1/4 ("4"), 3/5, 3/6, and 1 of length 7 ("8")
  sp = sorted(patterns, key=lambda s: len(s))
  len2_1_cf = sp[0]  # read: pattern of length 2 = digit "1" = segments c and f
  len3_7_acf = sp[1] 
  len4_4_bcdf = sp[2]  
  len5_235 = sp[3:6]  # read: list of patterns of length 5 (there are always 3), representing digits "2", "3", "5" (5 segments each)
  len6_069 = sp[6:9]  # dito for length 6
  len7_8_abcdefg = sp[9]

  # some utilities 

  def s_diff(s1:str, s2:str) -> str:
    return "".join(c for c in s1 if c not in s2)  

  def s_inter(s1:str, s2:str) -> str:
    return "".join(c for c in s1 if c in s2)

  def s_union(s1:str, s2:str) -> str:
    return "".join(set(s1+s2))

  def chars_in(s1:str, s2:str) -> bool:
    return sum(1 if c in s2 else 0 for c in s1) == len(s1)

  # one of many possible ways of deducting the mapping of the segment codes used in output to the correct segments
  
  mapping = {}
  # 1
  seg_a = s_diff(len3_7_acf, len2_1_cf)
  mapping[seg_a] = 'a'
  # 2 dg = "3" / len5 which includes ("1") - "7"
  seg_dg = s_diff([p for p in len5_235 if chars_in(len2_1_cf, p)][0], len3_7_acf)
  # 3 
  seg_bd = s_diff(len4_4_bcdf, len2_1_cf)
  # 4
  seg_d = s_inter(seg_dg, seg_bd)
  mapping[seg_d] = 'd'
  # 5
  seg_g = s_diff(seg_dg, seg_d)
  mapping[seg_g] = 'g'
  # 6
  seg_b = s_diff(seg_bd, seg_d)
  mapping[seg_b] = 'b'
  # 7
  seg_eg = s_diff(len7_8_abcdefg, s_union(len3_7_acf, len4_4_bcdf))
  # 8
  seg_e = s_diff(seg_eg, seg_g)
  mapping[seg_e] = 'e'
  # 9 f = "6" / len 6 which has only one of the segments of "1" (cf) - segments a,b,d,e,g
  seg_f = reduce(lambda p, seg: s_diff(p, seg), 
                 [seg_a, seg_b, seg_d, seg_e, seg_g], 
                 [p for p in len6_069 if len(s_inter(p, len2_1_cf)) == 1][0])
  mapping[seg_f] = 'f'
  # 10 
  seg_c = s_diff(len2_1_cf, seg_f)
  mapping[seg_c] = 'c'
  return mapping

# apply mapping to output to get correct segment codes
def translate(output, mapping):
  return "".join(mapping[o] for o in output)

# sort letters in a string (needed for looking up digits by segments string)
def s_sort(s:str) -> str:
  return "".join(sorted(s))

# digits by segments, it is important that the segment letters are sorted for each digit!
digits = dict((s, d) for d, s in enumerate(segments))

res = 0
for po in patterns_and_output:
  mapping = decode(po['patterns'])
  decoded_output = sum(digits[s_sort(translate(output, mapping))] * pow(10, i) for i, output in enumerate(reversed(po['output'])))
  # print(decoded_output)
  res += decoded_output

print("part 2:", res)

part 1: 539
part 2: 1084606


9) Height map

In [ ]:
heightmap = [[int(c) for c in line.strip()] for line in open('drive/MyDrive/AoC/2021/input09.txt')]

h = len(heightmap)
w = len(heightmap[0])

# in the following x, y are coordinates
# p (point), lp (low point), nb (neighbor) are tuples representing points with 2 coordinates
# so we can write x, y = p meaning x = p[0], y = p[1]
# low_points, nbs (neighbors), b (basin) are sets of tuples (points)

# this should be memoized / cached
def neighbors(p) -> list:
  x, y = p
  return [(xx, yy) 
          for yy in range(y-1, y+2) 
            for xx in range(x-1, x+2) 
              if ((xx == x) ^ (yy == y)) and xx in range(0, w) and yy in range(0, h)]

def height(p) -> int:
  x, y = p
  return heightmap[y][x]

# 1 - low points 

low_points = [
  (x, y) 
  for y in range(0, h)
    for x in range(0, w) 
      if len([p for p in neighbors((x, y)) if height(p) > height((x, y))]) == len(neighbors((x, y)))]

print("part 1:",
  sum(height(p) + 1 for p in low_points))

# part 2 - basins, using mutable sets

def basin(point) -> set:
  def extend_by(p):
    basin.add(p)
    for nb in neighbors(p):
      if nb not in basin and height(nb) in range(height(p), 9):
        extend_by(nb)
  basin = set()
  extend_by(point)
  return basin

basin_sizes = sorted([len(basin(lp)) for lp in low_points], reverse=True)
print("part 2 (mutable):", basin_sizes[0] * basin_sizes[1] * basin_sizes[2])

# part 2 - basins, using functional adding to sets (return new extended set)

from functools import reduce

def fbasin(point) -> set:
  def extend(basin, p) -> set:
    return reduce(lambda ext_basin, nb: extend(ext_basin, nb) if nb not in ext_basin and height(nb) in range(height(p), 9) else ext_basin, 
                  neighbors(p),      # iterable 'looped' over
                  set([*basin, p]))  # initial value, creates a new set (ext_basin), * explodes/unpacks the current basin
  return extend(set(), point)

fbasin_sizes = sorted([len(fbasin(lp)) for lp in low_points], reverse=True)
print("part 2 (functional):", fbasin_sizes[0] * fbasin_sizes[1] * fbasin_sizes[2])


part 1: 456
part 2 (mutable): 1047744
part 2 (functional): 1047744


10) Chunks surrounded by `(), [], {}, <>`


In [ ]:
lines = [l.strip() for l in open('drive/MyDrive/AoC/2021/input10.txt')]
opening = '([{<'
closing = ')]}>'
opened_by = dict((c, opening[i]) for i, c in enumerate(closing))
illegal_scores = {
    ')': 3,
    ']': 57,
    '}': 1197,
    '>': 25137
}
# 1 - corrupted lines, find first illegal char, score the line accordingly, sum up all line scores

def find_illegal_char(line):
  stack = []
  for ch in line:
    if (ch in opening):
      stack.append(ch)
    else:  # assume closed
      if not stack:
        return ch
      opened = stack.pop()
      if opened != opened_by[ch]:
        return ch
  return None

print("part 1:", sum(illegal_scores[il] if il else 0 for il in [find_illegal_char(line) for line in lines]))

# 2 - discard corrupted lines, complete the remaining lines and score each line for completion chars

closed_by = dict((o, closing[i]) for i, o in enumerate(opening))
completion_scores = dict((c, i+1) for i, c in enumerate(closing))

from functools import reduce

def score_completion(line):
  stack = []
  for ch in line:
    if (ch in opening):
      stack.append(ch)
    else:  # assume closed
      if not stack:
        return 0  # illegal
      opened = stack.pop()
      if opened != opened_by[ch]:
        return 0  # illegal
  return reduce(lambda score, o: 5 * score + completion_scores[closed_by[o]], list(reversed(stack)), 0)

scores = [score_completion(line) for line in lines]
filtered_scores = [s for s in scores if s > 0]

print("part 2:", sorted(filtered_scores)[int(len(filtered_scores)/2)])

part 1: 290691
part 2: 2768166558


11) Flashing octopuses

In [ ]:
grid = [[int(c) for c in l.strip()] for l in open('drive/MyDrive/AoC/2021/input11.txt')]
h = len(grid)
w = len(grid[0])

# this should be memoized / cached
def neighbors(p) -> list:
  x, y = p
  return [(xx, yy) 
          for yy in range(y-1, y+2) 
            for xx in range(x-1, x+2) 
              if (xx != x or yy != y) and xx in range(0, w) and yy in range(0, h)]

def get_energy(p) -> int:
  x, y = p
  return grid[y][x]

# mutates grid
def set_energy(p, energy):
  x, y = p
  grid[y][x] = energy

# increases point p in grid by 1 (mod 10) and returns True if p's energy is now 0 (flashing)
def inc_energy(p):
  x, y = p
  set_energy(p, (get_energy(p) + 1) % 10)
  return get_energy(p) == 0

def p():
  for row in grid:
    print("".join(str(x) for x in row))

# returns number of flashes in step
def step():
  flashing = [(x, y) for y in range(0, h) for x in range(0, w) if inc_energy((x, y))]
  flashed = flashing
  flashes = len(flashing)
  while flashing:
    increasing = [nb for p in flashing for nb in neighbors(p) if nb not in flashed]
    flashing = [p for p in increasing if get_energy(p) > 0 and inc_energy(p)]
    # p()
    flashed += flashing
    flashes += len(flashing)
  return flashes

# part 1 - how many flashes in 100 steps
flashes = 0
for i in range(0, 100):
#  print(i+1)
  flashes += step() 

print("part 1")
p()
print(flashes)

# part 2 - when do all octopuses flash simultaneously
steps = 100  # from part 1
while sum(sum(row) for row in grid) > 0:
  steps += 1
  step()

print("part 2:")
p()
print(steps)

part 1
1200211116
1239811111
1116581111
1775581111
1754651111
5545861111
8457082111
6790003122
5569003240
5557332200
1673
part 2:
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
279


In [ ]:
# functional

# this should be memoized / cached
def neighbors(grid, p) -> list:
  h = len(grid)
  w = len(grid[0])
  x, y = p
  return [(xx, yy) 
          for yy in range(y-1, y+2) 
            for xx in range(x-1, x+2) 
              if (xx != x or yy != y) and xx in range(0, w) and yy in range(0, h)]

def energy(grid, p):
  x, y = p
  return grid[y][x]

def p(grid):
  for row in grid:
    print("".join(str(e) for e in row))

# TODO Something does not match the (working) non-functional approach above. Too many flashes in part 1, endless iteration in part 2.

def inc(grid, points, inc_0s=False):
  new_grid = [[(energy + 1) % 10 if (x, y) in points and (inc_0s or energy > 0) else energy for x, energy in enumerate(row)] for y, row in enumerate(grid)]
  new_0s = [p for p in points if energy(new_grid, p) == 0]
  return (new_grid, new_0s)

# returns: new grid, num flashes
def step(gf):
  grid, flashes = gf
  h = len(grid)
  w = len(grid[0])
  increasing = [(x, y) for y in range(0, h) for x in range(0, w)]  # initially, increase energy for all points
  grid, flashing = inc(grid, increasing, inc_0s=True)
  flashed = flashing
  flashes += len(flashing)
  while flashing:
    increasing = [nb for p in flashing for nb in neighbors(grid, p) if nb not in flashed]
    grid, flashing = inc(grid, increasing, inc_0s=False)
    flashed += flashing
    flashes += len(flashing)
  return (grid, flashes)

# part 1 - how many flashes in 100 steps
initial_grid = [[int(c) for c in l.strip()] for l in open('drive/MyDrive/AoC/2021/input11_example.txt')]

from functools import reduce

g, f = reduce(lambda gf, i: step(gf), range(0, 100), (initial_grid, 0))

print('part 1:')
p(g)
print(f)

# part 2 - when do all octopuses flash simultaneously
steps = 100  # from part 1
while sum(sum(row) for row in g) > 0:
  steps += 1
  g, f = step((g, f))

print("part 2:")
p(g)
print(steps)

12) Caves graph

In [ ]:
import re
from functools import reduce

# graph = dict(node -> adjacent nodes) (nodes = caves)

# adds an edge to a graph by merging adjacent nodes for both nodes of given edge 
def add_edge(adj:dict, edge:tuple) -> dict:
  n1, n2 = edge
  return {**adj, **{n1: adj.get(n1, []) + [n2], n2: adj.get(n2, []) + [n1]}}  # functional merge of two dicts

graph = reduce(add_edge, 
               [re.fullmatch('([a-zA-Z]+)-([a-zA-Z]+)', line.strip()).groups() \
                for line in open('drive/MyDrive/AoC/2021/input12.txt')], 
               {})

def is_small(node):
  return node[0].islower()

def is_large(node):
  return node[0].isupper()

def walk1(node, path):
  # if node == 'end': print(path+[node])
  return 1 if node=='end' else sum(walk1(n, path+[node]) for n in graph[node] if is_large(n) or n not in path)

print('part 1:', walk1('start', []))

def walk2(node, small_twice, path):
  # if node == 'end': print(path+[node])
  return 1 if node=='end' \
           else sum(walk2(n, n if is_small(n) and n in path else small_twice, path+[node]) \
                    for n in graph[node] \
                    if is_large(n) or n not in path or (not small_twice and n != 'start'))

print('part 2:', walk2('start', None, []))

part 1: 4186
part 2: 92111


13) Folding transparent paper

In [ ]:
import re
from functools import reduce

dots = set(tuple(int(s) for s in line.strip().split(',')) for line in open('drive/MyDrive/AoC/2021/input13.txt') if re.match('^\d', line))
folds = [re.fullmatch('fold along ([xy])=(\d+)', line.strip()).groups() for line in open('drive/MyDrive/AoC/2021/input13.txt') if line.startswith('fold')]
print("dots :", len(dots))
print("folds:", folds)

def fold(dots, f):
  axis, s = f
  v = int(s)
  return set((x, y) if x < v else (2*v-x, y) for x, y in dots) if axis == 'x' else set((x, y) if y < v else (x, 2*v-y) for x, y in dots)

print('part 1:', len(fold(dots, folds[0])))

def p(code):
  code_size = reduce(lambda m, dot: (max(dot[0], m[0]), max(dot[1], m[1])), code, (0, 0))
  print("code size:", code_size)
  for y in range(0, code_size[1]+1):
    print("".join('#' if (x, y) in code else ' ' for x in range(0, code_size[0]+1)))

code = reduce(fold, folds, dots)

print('part 2:')
p(code)

dots : 903
folds: [('x', '655'), ('y', '447'), ('x', '327'), ('y', '223'), ('x', '163'), ('y', '111'), ('x', '81'), ('y', '55'), ('x', '40'), ('y', '27'), ('y', '13'), ('y', '6')]
part 1: 753
part 2:
code size: (38, 5)
#  # #### #    #### #  #   ## ###  #  #
#  #    # #    #    #  #    # #  # # # 
####   #  #    ###  ####    # #  # ##  
#  #  #   #    #    #  #    # ###  # # 
#  # #    #    #    #  # #  # # #  # # 
#  # #### #### #### #  #  ##  #  # #  #


14) Polymers

In [ ]:
import re
from functools import reduce

lines = [line.strip() for line in open('drive/MyDrive/AoC/2021/input14.txt')]
template = lines[0]
print('template:', template)
rules = dict(re.fullmatch('([A-Z][A-Z])\s*->\s*([A-Z])', lines[i]).groups() for i in range (2, len(lines)))
print('rules:', rules)

# brute force
def step(t):
  return t[0] + "".join(rules[t[i:i+2]] + t[i+1] for i in range(0, len(t)-1))

res = reduce(lambda iter, i: step(iter), range(0, 10), template)
# print(res)

def counts(seq):
  return reduce(lambda c, e: {**c, **{e: c.get(e, 0) + 1}}, seq, {})

c1 = counts(res)
print(c1)

print('part 1:', max(c1.values()) - min(c1.values()))

# counting single char occurrences (c) and pairs (p)

# helper: functional decrease value for key by delta, remove if <0
def dec(d, key, delta):
  return dict((k, v - delta) if k==key else (k, v) for k, v in d.items() if k != key or v > delta)

# applies rule for given pair (with occurrence of pair before step)
# cp = pair of dicts (char counts, pair counts)
def apply_rule(cp, pair_and_count):
  pair, count = pair_and_count
  c, p = cp
  p = dec(p, pair, count)  # pair is replaced by two new pairs (new char in the middle)
  new_c = rules[pair]
  new_p1 = pair[0] + new_c
  new_p2 = new_c + pair[1]
  return ({ **c, **{new_c: c.get(new_c, 0) + count}}, { **p, **{new_p1: p.get(new_p1, 0) + count, new_p2: p.get(new_p2, 0) + count}})

def step2(cp):
  c, p = cp
  return reduce(apply_rule, p.items(), cp)

# count chars and pairs, and iterate over them 40 times
c2, p2 = reduce(lambda cp, i: step2(cp), range(0, 40), (counts(template), counts([template[i:i+2] for i in range (0, len(template)-1)])))

print(c2, p2)
print('part 2:', max(c2.values()) - min(c2.values()))


template: PFVKOBSHPSPOOOCOOHBP
rules: {'FV': 'C', 'CP': 'K', 'FS': 'K', 'VF': 'N', 'HN': 'F', 'FF': 'N', 'SS': 'K', 'VS': 'V', 'BV': 'F', 'HC': 'K', 'BP': 'F', 'OV': 'N', 'BF': 'V', 'VH': 'V', 'PF': 'N', 'FC': 'S', 'CS': 'B', 'FK': 'N', 'VK': 'H', 'FN': 'P', 'SH': 'V', 'CV': 'K', 'HP': 'K', 'HO': 'C', 'NO': 'V', 'CK': 'C', 'VB': 'S', 'OC': 'N', 'NS': 'C', 'NF': 'H', 'SF': 'N', 'NK': 'S', 'NP': 'P', 'OO': 'S', 'NH': 'C', 'BC': 'H', 'KS': 'H', 'PV': 'O', 'KO': 'K', 'OK': 'H', 'OH': 'H', 'BH': 'F', 'NB': 'B', 'FH': 'N', 'HV': 'F', 'BN': 'S', 'ON': 'V', 'CB': 'V', 'CF': 'H', 'FB': 'S', 'KF': 'S', 'PS': 'P', 'OB': 'C', 'NN': 'K', 'KV': 'C', 'BK': 'H', 'SN': 'S', 'NC': 'H', 'PK': 'B', 'PC': 'H', 'KN': 'S', 'VO': 'V', 'FO': 'K', 'CH': 'B', 'PH': 'N', 'SO': 'C', 'KH': 'S', 'HB': 'V', 'HH': 'B', 'BB': 'H', 'SC': 'V', 'HS': 'K', 'SP': 'V', 'KB': 'N', 'VN': 'H', 'HK': 'H', 'KP': 'K', 'OP': 'F', 'CO': 'B', 'VP': 'H', 'OS': 'N', 'OF': 'H', 'KK': 'N', 'CC': 'K', 'BS': 'C', 'VV': 'O', 'CN': 'H', 'PB'

15) shortest (lowest weight) path search

In [ ]:
# Dijkstra shortest path from (0, 0) to target node
# We have a fully connected graph here consisting of h * w nodes where there are edges to 2-4 neighbors (2 in the corners, 3 at the borders, 4 in the middle).
# The node values can be viewed as distance from any of its neighbors

def dijkstra_dist(nodes, w1, h1, rep, goal):
  
  h = h1 * rep
  w = w1 * rep
  num_nodes = h * w
    
  # should be memoized ;)
  def neighbors(p) -> list:
    x, y = p
    return [(x, y)
            for x, y in [(x+1,y), (x,y+1), (x-1,y), (x,y-1)]
                if x in range(0, w) and x in range(0, h)]

  # init
  distances = {(0, 0): 0}  # point -> distance , default interpreted as infinite
  predecessors = {}  # point -> point , default interpreted as none
  nodes_to_visit = set((x, y) for y in range(0, h) for x in range(0, w))
  
  def risk(v):
    x, y = v
    dy = int(y/h1)
    dx = int(x/w1)
    diff = dx + dy
    return (nodes[(x-dx*w1, y-dy*w1)] + diff - 1) % 9 + 1

  def dist_update(u, v, dists, preds):
    alt = dists.get(u) + risk(v)
    if alt < dists.get(v, 999999):
      dists[v] = alt
      preds[v] = u

  while nodes_to_visit: # not empty
    if (len(nodes_to_visit) % 1000 == 0):
      print(len(nodes_to_visit))
    u = min(distances.items(), key=lambda d: d[1] if d[0] in nodes_to_visit else 999999)[0] \
      if len(nodes_to_visit) > num_nodes/2 \
      else min(nodes_to_visit, key=lambda n: distances.get(n, 999999)) 
    nodes_to_visit.remove(u)
    if u == goal:
      break
    for v in neighbors(u):
      if v in nodes_to_visit:
        dist_update(u, v, distances, predecessors)

  return distances[goal]

lines = [line.strip() for line in open('drive/MyDrive/AoC/2021/input15.txt')]
h = len(lines)
w = len(lines[0])
nodes = dict(((x, y), int(s))  for y, line in enumerate(lines) for x, s in enumerate(line))

print('part 1:', dijkstra_dist(nodes, w, h, 1, (w-1, h-1)))

### THIS IS REALLY SLOW - TAKES 2hrs FOR THE 250000 NODES OF PART 2 ### 
#print('part 2:', dijkstra_dist(nodes, w, h, 5, (5*w-1, 5*h-1)))


10000
9000
8000
7000
6000
5000
4000
3000
2000
1000
part 1: 523


16) Intcode

In [ ]:
from functools import reduce

# Tree classes for operators, operations, literals

# Added for part 2

class Operation:
  def __init__(self, name, operation):
    self.name = name
    self.operation = operation
  def name(self):
    return self.name
  def evaluate(self, operands):
    return self.operation(operands)
  def __str__(self):
    return self.name
  __repr__ = __str__

operations = [
  Operation('+', lambda operands: sum(o.evaluate() for o in operands)),
  Operation('*', lambda operands: reduce(lambda product, o: product * o.evaluate(), operands, 1)),
  Operation('min', lambda operands: min(o.evaluate() for o in operands)),
  Operation('max', lambda operands: max(o.evaluate() for o in operands)),
  None,  # literal
  Operation('>', lambda operands: 1 if operands[0].evaluate() > operands[1].evaluate() else 0),
  Operation('<', lambda operands: 1 if operands[0].evaluate() < operands[1].evaluate() else 0),
  Operation('==', lambda operands: 1 if operands[0].evaluate() == operands[1].evaluate() else 0)
]

# End added for part 2

class Literal:
  def __init__(self, version, val):
    self.version = version
    self.val = val
  def evaluate(self):
    return self.val
  def sum_version(self):
    return self.version
  def __str__(self):
    # return f'{self.val}[{self.version}]'  # part 1
    return f'{self.val}'  # part 2
  __repr__ = __str__

class Operator:
  def __init__(self, version, type_id, operands):
    self.version = version
    self.type_id = type_id
    self.operands = operands
  def evaluate(self):
    # return None  # part 1 (operations not yet known)
    return operations[self.type_id].evaluate(self.operands)
  def sum_version(self):
    return self.version + sum(o.sum_version() for o in self.operands)
  def __str__(self):
    # return f"op[{self.version},{self.type_id}]({', '.join(str(o) for o in self.operands)})"  # part 1 (operations not yet known)
    return f"{operations[self.type_id]}({', '.join(str(o) for o in self.operands)})"  # part 2
  __repr__ = __str__

# Helpers

# Read n bits as int, return with rest
def read_bits(bin_input, n):
  return (int(bin_input[:n], 2), bin_input[n:])

# Literal parsing: Lazy chunking of string into chunks of length n
# Yields pairs of chunk and rest after chunk
def chunked(string, chunk_len):
  rest = string
  while chunk_len <= len(rest):
    chunk = rest[:chunk_len]
    rest = rest[chunk_len:]
    yield (chunk, rest)

# Literal parsing: Lazily take from an iterable until predicate is fulfilled by
# the next element (which is then yielded as last one)
def takeuntil(pred, iterable):
  i = iter(iterable)
  n = next(i)
  while not pred(n):
    yield n
    n = next(i)
  yield n

# Literal parsing: Checks if a digit chunk is the last (first bit == 0)
# Input is a tuple (digit chunk, parsing rest), so the function can be used with takeuntil
# on a sequence produced by chunked
def is_last_digit(digit_and_rest) -> bool:
  digit, _ = digit_and_rest
  return digit[0] == '0'

# Literal parsing for use with reduce, appends the binary representation of a 
# digit to the previously parsed beginning of the literal;
# returns the new literal beginning with the rest after the chunk just added
def append_digit(literal_and_prev_rest, digit_and_rest):
  literal, _ = literal_and_prev_rest
  digit, rest = digit_and_rest
  return (literal + digit[1:len(digit)], rest)

# Parsing = recursive assembly of a tree of Operators and Literals

def parse_literal(version, bin_input, chunk_len):
  literal, rest = reduce(append_digit, takeuntil(is_last_digit, chunked(bin_input, chunk_len)), ('', ''))
  return (Literal(version, int(literal, 2)), rest)

def parse_operator_with_length_limit(version, type_id, bin_input):
  limit, rest = read_bits(bin_input, 15)
  packets, sub_rest = parse_packets(rest[:limit])
  assert sub_rest == ''
  return (Operator(version, type_id, packets), rest[limit:])

def parse_operator_with_operands_limit(version, type_id, bin_input):
  limit, rest = read_bits(bin_input, 11)
  packets, rest = parse_packets(rest, limit)
  return (Operator(version, type_id, packets), rest)

def parse_packets(bin_input, packet_limit=None):
  packets = []
  rest = bin_input
  while len(rest) >= 11 and (not packet_limit or len(packets) < packet_limit):
    version, rest = read_bits(rest, 3)
    type_id, rest = read_bits(rest, 3)
    if (type_id == 4):
      literal, rest = parse_literal(version, rest, 5)
      packets.append(literal)
    else: 
      len_type_id, rest = read_bits(rest, 1)
      operator, rest = parse_operator_with_length_limit(version, type_id, rest) \
                  if len_type_id==0 \
                  else parse_operator_with_operands_limit(version, type_id, rest)
      packets.append(operator)
  return packets, rest

hex_input = open('drive/MyDrive/AoC/2021/input16.txt').readline().strip()
# hex_input = '9C0141080250320F1802104A08'

packets, rest = parse_packets("".join(format(int(d, 16), '04b') for d in hex_input))
print("packets:", packets)
print("rest:", rest)
print("part 1, sum of packet versions:", sum(p.sum_version() for p in packets))
print("part 2, evaluation of packets:", [p.evaluate() for p in packets])


packets: [+(*(843, ==(813368, 813368)), 53943, *(<(34819, 34819), 7549566), max(11, 680, 58649361, 1258420029, 3613), +(7856267009332), *(63406121605, >(176, 64705)), 242, 119, min(658), min(62875, 2582, 96), *(716, <(1831, 1)), *(24693, >(424545, 15)), +(2, 3), *(<(234, 668770), 103076), *(<(227, 227), 3988), *(<(+(4, 15, 3), +(15, 9, 9)), 38844), *(9), min(3563, 72, 1885, 824940, 1022465), 94, *(89, ==(+(4, 15, 2), +(10, 11, 12))), *(+(9, 14, 15), +(7, 10, 6), +(11, 15, 11)), *(>(+(4, 9, 8), +(8, 10, 14)), 10), *(11889, >(77, 2640)), *(>(66148276326, 127), 18970730068), max(13, 139, 46), 9761, *(<(+(2, 14, 3), +(9, 3, 2)), 158452), *(==(64, 203), 497), +(6, 49, 1002099, 689786, 3646), +(12656205, 57796478963, 12634253), *(162, 47, 19, 98, 142), 142, min(595628, 174, 22302, 777), 3099, max(12, 1521), *(255, 86), max(780128, 33621, 59805, 177), *(<(8964053, 9), 359815), 42673, +(*(9, 10, 10), *(9, 4, 4), *(15, 13, 2)), max(+(max(*(max(min(min(min(*(max(max(+(max(min(max(min(+(+(*(*(30)

17) Trajectories

In [ ]:
import re
from itertools import takewhile
from math import sqrt

input = open('drive/MyDrive/AoC/2021/input17.txt').readline().strip()
print(input)
tdef = [int(g) for g in re.fullmatch('target area: x=(-?\d+)..(-?\d+), y=(-?\d+)..(-?\d+)', input).groups()]
t_min_x = tdef[0]
t_max_x = tdef[1]
t_min_y = tdef[2]
t_max_y = tdef[3]

def overshot_x(x):
  return x > t_max_x

def overshot_y(y):
  return y < t_min_y

def overshot(p):
  x, y = p
  return overshot_x(x) or overshot_y(y)

def simulate(x, y, vx, vy):
  while True:
    x += vx
    y += vy
    vx = max(vx-1, 0)
    vy -= 1
    yield x, y

def trajectory(init_vx, init_vy):
  return [(x, y) for x, y in takewhile(lambda p: not overshot(p), simulate(0, 0, init_vx, init_vy))]

def on_target(p):
  x, y = p
  return x in range(t_min_x, t_max_x+1) and y in range(t_min_y, t_max_y+1)

def t_on_target(trajectory):
  # we take while not overshot, so the last point must be on or before target
  return on_target(trajectory[-1]) 

def get_max_y(trajectory):
  return max(y for x, y in trajectory)

# Did not have much time and concentration for this. So just brute forced it,
# making sure to calculate all trajectories with initial velocities including
# - all the coordinates of the target area
#   (reaching the target in one step, covering all target fields) 
#   i. e. init_vx <= max x of the target, init_vy >= min y of the target
# - init_vx + (init_vx-1) + ... + 1 >= min x of the target (t_min_x), 
#   <=> init_vx(init_vx+1)/2 >= t_min_x
#   <=> init_vx^2 + init_vx - 2*t_min_x >= 0
#   <=> positive init_vx >= (-1 + sqrt(1 + 8*t_min_x))/2 
# - for upper bound of init_vy, just tried 100, 200 and
#   discovered that init_vy cannot be greater than 104 by looking at the output 
#   of printing all trajectories on target
# Then just calculated the trajectories for all initial velocities within these
# bounds and collected the max max(y) of all trajectories hitting the target

on_target = \
  [t for t in 
    [trajectory(init_vx, init_vy) 
      for init_vx in range(int((-1+sqrt(1+8*t_min_x))/2), t_max_x+1) 
      for init_vy in range(t_min_y, 105)]
    if t_on_target(t)]

print("max y if trajectory on target:", max(get_max_y(t) for t in on_target))
print("initial velocities on target:", len(on_target))



target area: x=206..250, y=-105..-57
max y if trajectory on target: 5460
initial velocities on target: 3618


18) Snailfish (Tree of pairs and literals)

In [ ]:
import math
from functools import reduce
from itertools import takewhile

class Node:
  def __init__(self):
    self.parent = None
  def find_num_left(self):
    return None if not self.parent else \
      (self.parent.find_num_left() if self == self.parent.left else self.parent.left.find_last_number())    
  def find_num_right(self):
    return None if not self.parent else \
      (self.parent.right.find_first_number() if self == self.parent.left else self.parent.find_num_right())  
  def add(self, right, debug=False):
# TODO if find_num_left()/right() above did not need self.parent, 
#  add() would not have to mutate the tree structure, i.e. no need for clone() and replace_with(),
#  also explosion and split could be implemented (as methods) to clone only nodes 
#  to be mutated and their ancestors (including root, and return the new root!)
    root = Pair(self.clone(), right.clone())
    if debug: print('after addition:', root)
    needs_reduce = True
    while needs_reduce:
# To reduce a snailfish number, you must repeatedly do the first action in 
# this list that applies to the snailfish number:
# - If any pair is nested inside four pairs, the leftmost such pair explodes. 
# - If any regular number is 10 or greater, the leftmost such regular number splits.
      deep_pair = root.find_first_pair_at_depth(4)
      if deep_pair:
# To explode a pair, the pair's left value is added to the first regular
# number to the left of the exploding pair (if any), and the pair's right 
# value is added to the first regular number to the right of the exploding pair (if any).
# Exploding pairs will always consist of two regular numbers. 
# Then, the entire exploding pair is replaced with the regular number 0.
        num_left = deep_pair.find_num_left()
        num_right = deep_pair.find_num_right()
        if num_left:
          num_left.val += deep_pair.left.val  # mutation
        if num_right:
          num_right.val += deep_pair.right.val  # mutation
        lit = Literal(0)
        deep_pair.replace_with(lit)  # mutation
        if debug: print('after explode: ', root)
      else:
        large_number = root.find_first_number_gte(10)
        if large_number:
# To split a regular number, replace it with a pair; 
# the left element of the pair should be the regular number divided by two and rounded down, 
# while the right element of the pair should be the regular number divided by two and rounded up.
          pair = Pair(Literal(math.floor(large_number.val/2)), Literal(math.ceil(large_number.val/2)))
          large_number.replace_with(pair)  # mutation
          if debug: print('after split:   ', root)
        else:
          needs_reduce = False
    return root
  def replace_with(self, node):
    # swaps out self for node by modifying node's ancestors left/right, returns self
    node.parent = self.parent
    if self.parent:
      if self == self.parent.left:
        self.parent.left = node
      else:
        self.parent.right = node
    return self

class Literal(Node):
  def __init__(self, val):
    Node.__init__(self)
    self.val = val
  def clone(self):
    return Literal(self.val)
  def find_first_number_gte(self, limit):
    return self if self.val >= limit else None
  def find_first_pair_at_depth(self, target_depth, depth):
    return None
  def find_first_number(self):
    return self
  def find_last_number(self):
    return self
  def magnitude(self):
    return self.val
  def __str__(self):
    return f'{self.val}'
  __repr__ = __str__

class Pair(Node):
  def __init__(self, left, right):
    Node.__init__(self)
    self.left = left
    self.right = right
    self.left.parent = self
    self.right.parent = self
  def clone(self):
    return Pair(self.left.clone(), self.right.clone())
  def find_first_number_gte(self, limit):
    return self.left.find_first_number_gte(limit) or self.right.find_first_number_gte(limit)
  def find_first_pair_at_depth(self, target_depth, depth=0):
    return self if depth == target_depth else \
      self.left.find_first_pair_at_depth(target_depth, depth+1) or \
      self.right.find_first_pair_at_depth(target_depth, depth+1)
  def find_first_number(self):
    return self.left.find_first_number()
  def find_last_number(self):
    return self.right.find_last_number()
  def magnitude(self):
    return 3*self.left.magnitude() + 2*self.right.magnitude()
  def __str__(self):
    return f'[{self.left}, {self.right}]'
  __repr__ = __str__

# returns parsing result and, if depth > 0, also the rest, assuming balanced brackets and no spaces
def parse(input, depth=0):
  char, rest = input[0], input[1:]
  if char == '[':
    left, rest = parse(rest, depth+1)
    right, rest = parse(rest[1:], depth+1)  # skip the ,
    pair = Pair(left, right)
    return (pair, rest[1:]) if depth > 0 else pair  # skip the ]; on root level, don't return the rest (assuming it is empty)
  else:
    # must be a digit
    rest_digits = ''.join(takewhile(lambda char: char in '0123456789', rest))
    lit = Literal(int(char + rest_digits))
    return (lit, rest[len(rest_digits):]) if depth > 0 else lit

parsed_input = [parse(line.strip()) for line in open('drive/MyDrive/AoC/2021/input18.txt')]
print("input size:", len(parsed_input))
print("part 1, magnitude of sum of all:     ", reduce(Node.add, parsed_input).magnitude())
print("part 2, largest m. of sum of any two:", 
      max(left.add(right).magnitude() for left in parsed_input for right in parsed_input if left != right))

input size: 100
part 1, magnitude of sum of all:      3305
part 2, largest m. of sum of any two: 4563


19) Beacon Scanner(s) (3D transformations)

In [ ]:
# Rotations for different orientations for beacon posns of a scan

# facing z, up y : rot 0
# -> f: z, up: x = rot y 0, rot z 90
# -> f: z, up: -y = rot y 0, rot z 180
# -> f: z, up: -x = rot y 0, rot z 270
# -> f: -z, up y = rot y 180
# -> f: -z, up x = rot y 180, rot z 270
# -> f: -z, up -y = rot y 180, rot z 180 = x180
# -> f: -z, up -x = rot y 180, rot z 90
# -> f: x, up y: rot y 90
# -> f: x, up z: rot y 90, rot x 90
# -> f: x, up -y: rot y 90, rot x 180
# -> f: x, up -z: rot y 90, rot x 270
# -> f: -x, up y: rot y 270
# -> f: -x, up z: rot y 270, rot x 270
# -> f: -x, up -y: rot y 270, rot x 180
# -> f: -x, up -z: rot y 270, rot x 90
# -> f: y, up -z: rot x 270
# -> f: y, up -x: rot x 270, rot y 90
# -> f: y, up z: rot x 270, rot y 180
# -> f: y, up x: rot x 270, rot y 270
# -> f: -y, up z: rot x 90
# -> f: -y, up x: rot x 90, rot y 90
# -> f: -y, up -z: rot x 90, rot y 180
# -> f: -y, up -x: rot x 90, rot y 270

import numpy as np
from numpy import array as a
from functools import reduce

r0 = a([[1,0,0],[0,1,0],[0,0,1]])

x90 = a([[1,0,0],[0,0,1],[0,-1,0]])
y90 = a([[0,0,-1],[0,1,0],[1,0,0]])
z90 = a([[0,-1,0],[1,0,0],[0,0,1]])

x180 = x90.dot(x90)
x270 = x180.dot(x90)
y180 = y90.dot(y90)
y270 = y180.dot(y90)
z180 = z90.dot(z90)
z270 = z180.dot(z90)

rotations = [
    z90, z180, z270, 
    y180, y180.dot(z270), x180, y180.dot(z90), 
    y90, y90.dot(x90), y90.dot(x180), y90.dot(x270),
    y270, y270.dot(x270), y270.dot(x180), y270.dot(x90), 
    x270, x270.dot(y90), x270.dot(y180), x270.dot(y270),
    x90, x90.dot(y90), x90.dot(y180), x90.dot(y270), 
    r0]  
    # r0 just for completeness (could just use the original array),
    # and last as we assume all scanners (except 0) are rotated, 

def p(scan):
  print('---')
  for pos in scan:
    print(pos)
  print('---')

def rotate(scan, rot):
  return [beacon.dot(rot) for beacon in scan]

def move(scan, offset):
  return [beacon + offset for beacon in scan]

# Returns offset l-r (= position of scanner r relative to l) and rotated and 
# moved scan r (such that the beacon positions in r are now relative to scanner l)
# if there are >= 12 overlapping beacons for scans l and any of the 24 possible
# orientations of r, None otherwise.
def offset_if_overlap(l, r):
  # TODO Use pre-computed rotations for all scans (except 0) and/or lazy rotation/yield and takeuntil  
  for rotation in rotations:
    rotated_r = rotate(r, rotation)
    diffs = reduce(lambda diffs, d: { **diffs, **{tuple(d): diffs.get(tuple(d), 0)+1} }, 
                   [beacon_l - beacon_r for beacon_l in l for beacon_r in rotated_r], 
                   {})
    offsets = [item[0] for item in diffs.items() if item[1] >= 12]
    if offsets:
      offset = a(offsets[0])
      return offset, move(rotated_r, offset)

# Returns the position of scanner r and a merged set of beacon positions of 
# scans l and r, relative to scanner l, if at least 12 beacon positions between 
# l and (a possibly rotated and moved) r overlap, None otherwise
def merge(l, r):
  offset_and_moved_r = offset_if_overlap(l, r)
  if offset_and_moved_r:
    offset, moved_r = offset_and_moved_r
    s = set([tuple(v) for v in l])
    s.update([tuple(v) for v in moved_r])
    return offset, [a(v) for v in s]  # offset = position of right scanner relative to left

scans = []
for line in open('drive/MyDrive/AoC/2021/input19.txt'):
  if line.startswith('---'):
    # scan = None
    scan = []
  elif line.strip():
    beacon = [int(s.strip()) for s in line.split(',')]
    # if not scan:
    #  scan = a([beacon])
    # else:
    #  scan = np.vstack([scan, beacon])
    scan.append(a(beacon))
  else: 
    scans.append(scan)
scans.append(scan)

print("scans:", len(scans))
merged = scans.pop(0)
# print(len(scans))

scanners = [a([0,0,0])]

# TODO This is quite slow, maybe a recursive divide/conquer approach, first 
# merging pairs, then merging the merged pairs, etc. would be faster
# (Mind the offsets/scanner positions!)
while scans:
  scan = scans.pop(0)
  scanner_r_and_merged = merge(merged, scan)
  if scanner_r_and_merged:
    scanner_r, merged = scanner_r_and_merged
    scanners.append(scanner_r)
    # print(len(scans))
  else:
    scans.append(scan) 
  
print("part 1, number of beacons:", len(merged))
print("part 2, max Manhattan distance between scanners:", max(sum([abs(d) for d in s1-s2]) for i, s1 in enumerate(scanners) for j, s2 in enumerate(scanners) if j>i))

scans: 36
part 1, number of beacons: 408
part 2, max Manhattan distance between scanners: 13348


20) Image enhancement (Game of Life)

In [ ]:
lines = [line.strip() for line in open('drive/MyDrive/AoC/2021/input20.txt')]

# first line are the 'enhancement' (transformation) rules, 511 values 0/1 for
# every possible combination of 3x3 pixels (0/1) (2^9 or 1 << 9)
enh = [1 if c == '#' else 0 for c in lines[0]]
print('enhancement rules:', enh)

# We only store 1-pixels in a dict with key (x,y) and value 1, and regard 0 as 'default' 
# That way, the number of pixels is just the sum of values of the dict
# Also, that way no modulo arithmetic is needed for the area_code (see below),
# ... but only if 1-pixels cannot appear out of nothing (enhancement of 0 is 1)...
init_img = dict(((x, y), 1) for y, row in enumerate(lines[2:]) for x, col in enumerate(row) if col == '#')

# Initial width and height
init_w = max(p[0] for p in init_img.keys()) + 1
init_h = max(p[1] for p in init_img.keys()) + 1

def get_pixel(img, x, y): 
  return img.get((x, y), 0)  # 0 is the default as there is never a key with value 0

# In the following, we extend the initial width and height by a margin on both
# sides. To simulate an infinite image size, we just need to choose the margin 
# large enough for all iterations (the image, consisting of 1-pixels, will 
# become larger, typically by 2 in width and height (left/right and above/under)
# in every iteration.
# The assumption for the margin is that all pixels inside the margin are initially 0
# That is also the case should the margin be extended.

def wrap_x(x, m):
  return (x + m) % (init_w + 2*m) - m 

def wrap_y(y, m):
  return (y + m) % (init_h + 2*m) - m

# Area code around x, y by enumerations of pixel values around (x, y) as follows
# y-1 [8, 7, 6]
# y   [5, 4, 3]
# y+1 [2, 1, 0]
#     x-1 x x+1
def area_code(img, x, y, m):
  return sum((1 << i) * p for i, p in enumerate(
      [get_pixel(img, xx, yy)
        for yy in [wrap_y(y+1,m), y, wrap_y(y-1,m)] 
        for xx in [wrap_x(x+1,m), x, wrap_x(x-1,m)]]))

def p(img, m):
  for y in range(-m, init_h+m):
    print(''.join(['#' if get_pixel(img,x,y) == 1 else '.' for x in range(-m, init_w+m)]))

def enhance(img, m):
  return dict([((x, y), e) for (x, y, e) in 
               [(x, y, enh[area_code(img, x, y, m)]) 
                for y in range(-m, init_h+m) 
                for x in range(-m, init_w+m)] 
               if e])  # only pixels enhanced to 1
     
iterations=50
img=init_img
for i in range(0, iterations): # part 1: 2 iterations, part 2: 50 iterations
  # print(i+1)
  # !!! the margin increase every second step does only work if the margin 
  # (or an extension of the margin) is initially empty as is the case with
  # flipping empty (0) and fully occupied (511) areas (enh[0]==1, enh[511]==0)
  # if this produces wrong results, just hardcode the margin to 50 or more (slower!)
  img = enhance(img, (int(i/2)+1)*2)  

# p(img, 50)

print(f'Number of #-pixels after {iterations} itertions:', sum(img.values()))

enhancement rules: [1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,

21) Dirac Dice

In [ ]:
init_positions = [int(line.strip()[-1:]) for line in open('drive/MyDrive/AoC/2021/input21.txt')]
init_positions = tuple(p if p > 0 else 10 for p in init_positions)
# init_positions = [4, 8]

print("players starting at:", init_positions)

# part 1
scores = [0, 0]
roll = 100 # so next is 1
iteration = 0

def next_roll_1(last_roll):
  return (last_roll % 100 + 1)  # last_roll + 1 but need 1..100, so +1, -1, %, + 1

#print(iteration, 'x', last_roll, positions, scores)

positions = [p for p in init_positions]
while max(scores) < 1000 or iteration % 3 != 0:
  player = int((iteration % 6) / 3)
  roll = roll % 100 + 1
  positions[player] = (positions[player] + roll - 1) % 10 + 1
  iteration += 1
  if iteration % 3 == 0: scores[player] += positions[player]
  # print(iteration, player, last_roll, positions, scores)

print("part 1:", iteration, "iterations, scores:", scores, "=>", min(scores) * iteration)

# part 2

# possible rolls and distribution over 27 universes for 3 rolls
# i.e. 111 = 3 in universe 0
#      112 = 4 in universe 1
#      113 = 5 in universe 2
#      121 = 4 in universe 3
# ...
#      333 = 9 in universe 26 

# instead of splitting up in 3 universes with equal frequency distribution (1) after each roll,
# we can split up in 7 scores after every 3 rolls, with different distributions
# so for 6 rolls there are 7^2 = 49 possible outcomes with different distributions, instead of 3^6 = 729 with equal distributionn, etc.
# when recursing down, we just need to multiply the frequencies for each branch

# returns [#universes player 1 wins, #universes player 2 wins]
def iterate(player, positions, scores):
  if scores[0] >= 21:
    return (1,0)
  elif scores[1] >= 21:
    return (0,1)
  else: 
    wins = [0,0]
    p0, p1 = positions
    s0, s1 = scores
    for roll, frequency in [(3,1),(4,3),(5,6),(6,7),(7,6),(8,3),(9,1)]:
      new_position = (positions[player] + roll - 1) % 10 + 1
      new_score = scores[player] + new_position
      new_positions = [p0, p1]
      new_positions[player] = new_position
      new_scores = [s0, s1]
      new_scores[player] = new_score
      new_wins = iterate(player^1, new_positions, new_scores)
      wins[0] += frequency*new_wins[0]
      wins[1] += frequency*new_wins[1]
    return wins

wins = iterate(0, init_positions, [0,0])
print("part 2, #universes in which player 1 and 2 win with score >= 21:", wins, "with max:", max(wins))


players starting at: (9, 6)
part 1: 1098 iterations, scores: [915, 1000] => 1004670
part 2, #universes in which player 1 and 2 win with score >= 21: [492043106122795, 267086464416104] with max: 492043106122795


22) Reactor init

In [ ]:
import re
all_instructions = [
  [1 if g == 'on' else 0 if g == 'off' else int(g) for g in re.fullmatch('(on|off) x=(-?\d+)..(-?\d+),y=(-?\d+)..(-?\d+),z=(-?\d+)..(-?\d+)', line.strip()).groups()] 
  for line in open('drive/MyDrive/AoC/2021/input22.txt')]

init_instructions = [i for i in all_instructions if i[1] >= -50 and i[2] <= 50]
outside_instructions = [i for i in all_instructions if i[1] < -50 or i[2] > 50]

# part 1, brute force every single cube
grid = dict([((x, y, z), c) 
             for c, x1, x2, y1, y2, z1, z2 in init_instructions
             for x in range(x1, x2+1) 
             for y in range(y1, y2+1) 
             for z in range(z1, z2+1)])

on_init = sum(grid.values())

print('part 1, cubes turned on in init region: ', sum(grid.values()))


part 1, cubes turned on in init region:  576028


In [ ]:
# part 2

# 1) check intersections init/outside (OPTIONAL, not really needed)

# pre: rx1, rx2 sorted ascendingly for x in 1,2
def range_intersection(r11, r12, r21, r22):
  if r21 <= r12 and r22 >= r11:
    return max(r11, r21), min(r12, r22)

intersections_init_outside = []

for i, init in enumerate(init_instructions):
  ic, ix1, ix2, iy1, iy2, iz1, iz2 = init
  for o, outside in enumerate(outside_instructions):
    oc, ox1, ox2, oy1, oy2, oz1, oz2 = outside
    rix, riy, riz = range_intersection(ix1,ix2,ox1,ox2), range_intersection(iy1,iy2,oy1,oy2), range_intersection(iz1,iz2,oz1,oz2)
    if rix and riy and riz:
      intersections_init_outside.append((i, o, init, outside, rix, riy, riz))

print(f"part 2, {len(intersections_init_outside)} intersections of init and outside instructions:", intersections_init_outside)
if intersections_init_outside:
  raise(f"Assumed no intersections of init and outside instructions but got {intersections_init_outside}")

# 2) Assuming none above, now check intersections outside/outside (OPTIONAL, not really needed)

intersections_outside = []
intersecting_instructions = set()
for o1, outside1 in enumerate(outside_instructions): 
  o1c, o1x1, o1x2, o1y1, o1y2, o1z1, o1z2 = outside1
  for o2, outside2 in enumerate(outside_instructions):
    if o2 > o1:
      o2c, o2x1, o2x2, o2y1, o2y2, o2z1, o2z2 = outside2
      rix, riy, riz = range_intersection(o1x1,o1x2,o2x1,o2x2), range_intersection(o1y1,o1y2,o2y1,o2y2), range_intersection(o1z1,o1z2,o2z1,o2z2)
      if rix and riy and riz:
        intersecting_instructions.add(o1)
        intersecting_instructions.add(o2)
        intersections_outside.append((o1, o2, outside1, outside2, rix, riy, riz))

print(f"part 2, {len(intersections_outside)} intersections among {len(intersecting_instructions)} of {len(outside_instructions)} outside instructions")

part 2, 0 intersections of init and outside instructions: []
part 2, 1606 intersections among 400 of 400 outside instructions


In [ ]:
# 3) Assuming there are intersections among the outside regions, 
# we cannot just add up the number of cubes in regions that are switched on.

# We need to split up the whole reactor in as many sections per axis as there 
# are distinct coordinates, forming a number of non-overlapping regions, which 
# are then affected by the reboot procedure 

from functools import reduce
xyz_vals = reduce(lambda vals, instr: (set([*vals[0], instr[1], instr[2]+1]),   # append x1, x2+1
                                       set([*vals[1], instr[3], instr[4]+1]),   # append y2, y2+1
                                       set([*vals[2], instr[5], instr[6]+1])),  # append z2, z2+1
                  outside_instructions, 
                  (set(), set(), set()))
x_vals, y_vals, z_vals = sorted(xyz_vals[0]), sorted(xyz_vals[1]), sorted(xyz_vals[2])

print(f"part 2, found {len(x_vals)}, {len(y_vals)}, {len(z_vals)} distinct x, y, z outside coordinates")

part 2, found 798, 797, 795 distinct x, y, z outside coordinates


In [ ]:
# compaction/de-compaction functions for indexes into coordinate arrays to save some RAM on Colab

def index_key(ix, iy, iz):
  return ix << 20 | iy << 10 | iz

def key_indexes(key):
  iz = key & 1023
  iy = (key >> 10) & 1023
  ix = key >> 20
  return ix, iy, iz

In [ ]:
# 4) (needs a lot of memory!)
# Go through the outside instructions and execute the command (on/1 or off/0)
# for all affected non-overlapping regions.
# We are not counting cubes, yet, but keep the resulting region state (0 or 1)
# for the x, y, and z indexes into the distinct coordinate lists.

region_states = {}  #  (ix, iy, iz) -> 0/1
on_all = on_init
for o, instr in enumerate(outside_instructions):
  command, x1, x2, y1, y2, z1, z2 = instr
  rx = range(x_vals.index(x1), x_vals.index(x2+1))
  ry = range(y_vals.index(y1), y_vals.index(y2+1))
  rz = range(z_vals.index(z1), z_vals.index(z2+1))
  for ix in rx:
    for iy in ry:
      for iz in rz:
        if command:
          region_states[index_key(ix,iy,iz)] = command  # `cube_states[(x, y, z)] = command` would crash Colab (RAM!)
        else:
          region_states.pop(index_key(ix,iy,iz), 0)  # save some more RAM (slower!)
  if ((o+1) % 100 == 0):
    print("processed outside instruction", o+1, "of", len(outside_instructions))

print("part 2, processed", o+1, "outside instructions:", len(region_states), "outside regions switched on")


processed outside instruction 100 of 400
processed outside instruction 200 of 400
processed outside instruction 300 of 400
processed outside instruction 400 of 400
part 2, processed 400 outside instructions: 136249777 outside regions switched on


In [ ]:
# 5) (needs a lot of memory!) 
# Iterate over non-overlapping regions that were switched on, look up the
# coordinates and add up the number of cubes.

on_all = on_init  # from part 1
for i, (key, command) in enumerate(region_states.items()):
  if i % 10000000 == 0: print("processed", i, "regions switched on, found", on_all, "cubes switched on")
  # if command == 1:
  ix, iy, iz = key_indexes(key)
  on_all += (x_vals[ix+1]-x_vals[ix]) * (y_vals[iy+1]-y_vals[iy]) * (z_vals[iz+1]-z_vals[iz])

# input22_example.txt: 2758514936282235
# input22.txt:         1387966280636636
print("part 2, total cubes switched on:", on_all)

processed 0 regions switched on, found 576028 cubes switched on
processed 10000000 regions switched on, found 100763334202924 cubes switched on
processed 20000000 regions switched on, found 205262735565016 cubes switched on
processed 30000000 regions switched on, found 304535214882454 cubes switched on
processed 40000000 regions switched on, found 409480316808415 cubes switched on
processed 50000000 regions switched on, found 520037862208949 cubes switched on
processed 60000000 regions switched on, found 617520244206419 cubes switched on
processed 70000000 regions switched on, found 708177873965042 cubes switched on
processed 80000000 regions switched on, found 809878049749056 cubes switched on
processed 90000000 regions switched on, found 912938786643563 cubes switched on
processed 100000000 regions switched on, found 1010250994392372 cubes switched on
processed 110000000 regions switched on, found 1105787719009685 cubes switched on
processed 120000000 regions switched on, found 12047

23) Amphipods

In [ ]:
# Part 1
# Skip parsing...
#############
#...........#
###D#C#A#B###
  #D#C#B#A#
  #########
energy = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}
target_col = {'A': 2, 'B': 4, 'C': 6, 'D': 8}
neighbors = {
    (0,0): [(1,0)],
    (1,0): [(0,0), (2,0)],
    (2,0): [(1,0), (3,0), (2,1)],
    (2,1): [(2,0), (2,2)],
    (2,2): [(2,1)],
    (3,0): [(2,0), (4,0)],
    (4,0): [(3,0), (5,0), (4,1)],
    (4,1): [(4,0), (4,2)],
    (4,2): [(4,1)],
    (5,0): [(4,0), (6,0)],
    (6,0): [(5,0), (7,0), (6,1)],
    (6,1): [(6,0), (6,2)],
    (6,2): [(6,1)],
    (7,0): [(6,0), (8,0)],
    (8,0): [(7,0), (9,0), (8,1)],
    (8,1): [(8,0), (8,2)],
    (8,2): [(8,1)],
    (9,0): [(8,0), (10,0)],
    (10,0): [(9,0)]
}

def on_target(pos, p, pod):
  return p[0] == target_col[pod] and (p[1] == 2 or p[1] == 1 and pos.get((p[0], 2), None) == pod)

def in_hall(p):
  return p[1] == 0 and p[0] not in target_col.values()

target_pos = {(2,1): 'A', (2,2): 'A', (4,1): 'B', (4,2): 'B', (6,1): 'C', (6,2): 'C', (8,1): 'D', (8,2): 'D'}

# TODO replace this global variable by return value from recursion!
min_target_e = [0]

# TODO the list comprehensions in various places below seem slower than for-loops with list.append

# return [(next_e, next_pos)] sorted by next_e
def find_next_positions(e, pos):

  # return: [(energy, next_p)]
  def find_paths(e, p, pod, from_hall, visited):
    next_e = e + energy[pod]  
    if (min_target_e[0] > 0 and next_e >= min_target_e[0]):  # > would find multiple solutions per min energy step
      return []
    for next_p in neighbors[p]:
      if next_p not in visited and next_p not in pos and on_target(pos, next_p, pod):
        return [(next_e, next_p)]  # shortcut - ignore neighbors if we can move to target directly
    res = []
    for next_p in neighbors[p]:
      if next_p not in visited and next_p not in pos:  # not in pos = not occupied
        if not from_hall and in_hall(next_p):
          res += [(next_e, next_p)]
        res += find_paths(next_e, next_p, pod, from_hall, visited + [next_p])
    return res

  def reduce_paths(pod, paths):
    # cut off hall positions if a pod can go straight to target
    return [(next_e, next_p) for next_e, next_p in paths if on_target(pos, next_p, pod)] or paths

  def next_pos(p, pod, next_p):
    res = dict(pos)
    res.pop(p, None)
    res[next_p] = pod
    return res

  next_positions = [(next_e, next_pos(p, pod, next_p))
    for p, pod in pos.items() 
    for next_e, next_p in reduce_paths(pod, find_paths(e, p, pod, in_hall(p), [p])) 
    if not on_target(pos, p, pod)]

  next_on_target = [np for np in next_positions if np == target_pos]
  next_other = [np for np in next_positions if np != target_pos]
  return [*sorted(next_on_target, key=lambda pair: pair[0]), *sorted(next_other, key=lambda pair: pair[0])]

def iterate_d(e, pos, depth):

  # extra function for print side-effect so we see the moves
  def flatten(iteration, prev_pos):
    fl = [next_pos for result in iteration for next_pos in result]
    if fl: print(depth+1, prev_pos)
    return fl

  if pos == target_pos:
    print("Found solution for energy:", e, "at depth:", depth)
    min_target_e[0] = e
    return [(e, pos)]
  return [flatten(iterate_d(next_e, next_pos, depth+1), next_pos) for next_e, next_pos in find_next_positions(e, pos)]

# inspired by https://github.com/davearussell/advent2021/blob/master/day23/solve.py
def iterate_b(pos0):
  q = [(0, pos0)]
  min_energy = 0
  depth = 0
  while q:
    depth += 1
    new_positions = [] 
    # find next positions 
    for e, pos in q:
      for next_e, next_pos in find_next_positions(e, pos):
        if next_pos == target_pos:
          print("Found solution for energy:", next_e, "at depth:", depth+1)
          min_energy = next_e if min_energy == 0 or next_e < min_energy else min_energy
        else:
          new_positions.append((next_e, next_pos))
    # eliminate duplicates, keep those with lowest energies
    energies = {}
    for e, pos in new_positions:
      key = tuple(sorted(pos.items(), key=lambda item:item[0]))
      if e < energies.get(key, 1000000):
        energies[key] = e
    q = [(e, dict(k)) for k, e in energies.items()]
  return min_energy
  

#############
#...........#
###D#C#A#B###
  #D#C#B#A#
  #########
init_pos = {(2,1): 'D', (2,2): 'D', (4,1): 'C', (4,2): 'C', (6,1): 'A', (6,2): 'B', (8,1): 'B', (8,2): 'A'}

#iterate_d(0, init_pos, 0)
iterate_b(init_pos)


Found solution with energy: 19160 at depth: 12
Found solution with energy: 19160 at depth: 12
Found solution with energy: 19162 at depth: 12
Found solution with energy: 19224 at depth: 12
Found solution with energy: 19162 at depth: 12
Found solution with energy: 19206 at depth: 12
Found solution with energy: 19198 at depth: 12
Found solution with energy: 19178 at depth: 12
Found solution with energy: 19178 at depth: 12
Found solution with energy: 19218 at depth: 12
Found solution with energy: 20538 at depth: 12
Found solution with energy: 20538 at depth: 12
Found solution with energy: 20538 at depth: 12
Found solution with energy: 20538 at depth: 12
Found solution with energy: 23156 at depth: 12
Found solution with energy: 23156 at depth: 12
Found solution with energy: 25156 at depth: 12
Found solution with energy: 25156 at depth: 12
Found solution with energy: 23160 at depth: 13
Found solution with energy: 19178 at depth: 13
Found solution with energy: 19198 at depth: 13
Found solutio

19160

In [ ]:
# Part 2
# Skip parsing...
#############
#...........#
###D#C#A#B###
  #D#C#B#A#     
  #D#B#A#C#
  #D#C#B#A#
  #########
energy = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}
target_col = {'A': 2, 'B': 4, 'C': 6, 'D': 8}
neighbors = {
    (0,0): [(1,0)],
    (1,0): [(0,0), (2,0)],
    (2,0): [(1,0), (3,0), (2,1)],
    (2,1): [(2,0), (2,2)],
    (2,2): [(2,1), (2,3)],
    (2,3): [(2,2), (2,4)],
    (2,4): [(2,3)],
    (3,0): [(2,0), (4,0)],
    (4,0): [(3,0), (5,0), (4,1)],
    (4,1): [(4,0), (4,2)],
    (4,2): [(4,1), (4,3)],
    (4,3): [(4,2), (4,4)],
    (4,4): [(4,3)],
    (5,0): [(4,0), (6,0)],
    (6,0): [(5,0), (7,0), (6,1)],
    (6,1): [(6,0), (6,2)],
    (6,2): [(6,1), (6,3)],
    (6,3): [(6,2), (6,4)],
    (6,4): [(6,3)],
    (7,0): [(6,0), (8,0)],
    (8,0): [(7,0), (9,0), (8,1)],
    (8,1): [(8,0), (8,2)],
    (8,2): [(8,1), (8,3)],
    (8,3): [(8,2), (8,4)],
    (8,4): [(8,3)],
    (9,0): [(8,0), (10,0)],
    (10,0): [(9,0)]
}

def on_target(pos, p, pod):
  return p[0] == target_col[pod] and (
      p[1] == 4 or 
      p[1] == 3 and pos.get((p[0], 4), None) == pod or
      p[1] == 2 and pos.get((p[0], 3), None) == pod and pos.get((p[0], 4), None) == pod or
      p[1] == 1 and pos.get((p[0], 2), None) == pod and pos.get((p[0], 3), None) == pod and pos.get((p[0], 4), None) == pod)

def in_hall(p):
  return p[1] == 0 and p[0] not in target_col.values()

target_pos = {(2,1): 'A', (2,2): 'A', (2,3): 'A', (2,4): 'A', (4,1): 'B', (4,2): 'B', (4,3): 'B', (4,4): 'B', 
              (6,1): 'C', (6,2): 'C', (6,3): 'C', (6,4): 'C', (8,1): 'D', (8,2): 'D', (8,3): 'D', (8,4): 'D'}

# TODO replace this global variable by return value from recursion!
min_target_e = [0]

# TODO the list comprehensions in various places below seem slower than for-loops with list.append

# return [(next_e, next_pos)] sorted by next_e
def find_next_positions(e, pos):

  # return: [(energy, next_p)]
  def find_paths(e, p, pod, from_hall, visited):
    next_e = e + energy[pod]  
    if (min_target_e[0] > 0 and next_e >= min_target_e[0]):  # > would find multiple solutions per min energy step
      return []
    for next_p in neighbors[p]:
      if next_p not in visited and next_p not in pos and on_target(pos, next_p, pod):
        return [(next_e, next_p)]  # shortcut - ignore neighbors if we can move to target directly
    res = []
    for next_p in neighbors[p]:
      if next_p not in visited and next_p not in pos:  # not in pos = not occupied
        if not from_hall and in_hall(next_p):
          res += [(next_e, next_p)]
        res += find_paths(next_e, next_p, pod, from_hall, visited + [next_p])
    return res

  def reduce_paths(pod, paths):
    # cut off hall positions if a pod can go straight to target
    return [(next_e, next_p) for next_e, next_p in paths if on_target(pos, next_p, pod)] or paths
  
  def next_pos(p, pod, next_p):
    res = dict(pos)
    res.pop(p, None)
    res[next_p] = pod
    return res

  next_positions = [(next_e, next_pos(p, pod, next_p))
    for p, pod in pos.items() 
    for next_e, next_p in reduce_paths(pod, find_paths(e, p, pod, in_hall(p), [p])) 
    if not on_target(pos, p, pod)]

  next_on_target = [np for np in next_positions if np == target_pos]
  next_other = [np for np in next_positions if np != target_pos]
  return [*sorted(next_on_target, key=lambda pair: pair[0]), *sorted(next_other, key=lambda pair: pair[0])]

def iterate_d(e, pos, depth):

  # extra function for print side-effect so we see the moves
  def flatten(iteration, prev_pos):
    fl = [next_pos for result in iteration for next_pos in result]
    if fl: print(depth+1, prev_pos)
    return fl

  if pos == target_pos:
    print("Found solution for energy:", e, "at depth:", depth)
    min_target_e[0] = e
    return [(e, pos)]
  return [flatten(iterate_d(next_e, next_pos, depth+1), next_pos) for next_e, next_pos in find_next_positions(e, pos)]

# inspired by https://github.com/davearussell/advent2021/blob/master/day23/solve.py
def iterate_b(pos0):
  q = [(0, pos0)]
  min_energy = 0
  depth = 0
  while q:
    depth += 1
    new_positions = [] 
    # find next positions 
    for e, pos in q:
      for next_e, next_pos in find_next_positions(e, pos):
        if next_pos == target_pos:
          print("Found solution for energy:", next_e, "at depth:", depth+1)
          min_energy = next_e if min_energy == 0 or next_e < min_energy else min_energy
        else:
          new_positions.append((next_e, next_pos))
    # eliminate duplicates, keep those with lowest energies
    energies = {}
    for e, pos in new_positions:
      key = tuple(sorted(pos.items(), key=lambda item:item[0]))
      if e < energies.get(key, 1000000):
        energies[key] = e
    q = [(e, dict(k)) for k, e in energies.items()]
  return min_energy


#############
#...........#
###D#C#A#B###
  #D#C#B#A#     
  #D#B#A#C#
  #D#C#B#A#
  #########
init_pos = {(2,1): 'D', (2,2): 'D', (2,3): 'D', (2,4): 'D', (4,1): 'C', (4,2): 'C', (4,3): 'B', (4,4): 'C', 
            (6,1): 'A', (6,2): 'B', (6,3): 'A', (6,4): 'B', (8,1): 'B', (8,2): 'A', (8,3): 'C', (8,4): 'A'}

#min_target_e[0] = 47237  # found 47236 in previous attempt
#iterate_d(0, init_pos, 0)

iterate_b(init_pos)

Found solution for energy: 47232 at depth: 24
Found solution for energy: 47232 at depth: 24
Found solution for energy: 48464 at depth: 24
Found solution for energy: 47466 at depth: 24
Found solution for energy: 48266 at depth: 24
Found solution for energy: 47466 at depth: 24
Found solution for energy: 47486 at depth: 24
Found solution for energy: 47784 at depth: 24
Found solution for energy: 47704 at depth: 24
Found solution for energy: 47744 at depth: 24
Found solution for energy: 47232 at depth: 25
Found solution for energy: 47232 at depth: 25
Found solution for energy: 48266 at depth: 25
Found solution for energy: 47466 at depth: 25
Found solution for energy: 48466 at depth: 25
Found solution for energy: 48866 at depth: 25
Found solution for energy: 48266 at depth: 25
Found solution for energy: 47466 at depth: 25
Found solution for energy: 47486 at depth: 25
Found solution for energy: 47784 at depth: 25
Found solution for energy: 47704 at depth: 25
Found solution for energy: 47744 a

47232

24) MONAD ALU programm
* inp a - Read an input value and write it to variable a.
* add a b - Add the value of a to the value of b, then store the result in variable a. 
* mul a b - Multiply the value of a by the value of b, then store the result in variable a.
* div a b - Divide the value of a by the value of b, truncate the result to an integer, then store the result in variable a. (Here, "truncate" means to round the value toward zero.)
* mod a b - Divide the value of a by the value of b, then store the remainder in variable a. (This is also called the modulo operation.)
* eql a b - If the value of a and b are equal, then store the value 1 in variable a. Otherwise, store the value 0 in variable a.

a = w, x, y, z
b = w, x, y, z or number

we expect only non-zero digits (1-9) as parameters to inp

part 1) find the largest 14 digit number (without 0-digits) for which the program ends with z=0

part 2) find the smallest 14 digit number (without 0-digits) for which the program ends with z=0

In [50]:
# idea -> parse and simplify program and print a representation as formula(s)
# then iterate and eval to find max/min input producing zero in z register

class Operation():
  def __init__(self):
    pass
  def eval(self, inputs):
    return None
  def simplify(self):
    return self
  def __eq__(self, o: object) -> bool:
    return type(self) == type(o)
  def __ne__(self, o: object) -> bool:
    return type(self) != type(o)

class UnaryOperation(Operation):
  def __init__(self, left):
    super().__init__()
    self.left = left
  def __eq__(self, o: object) -> bool:
    return type(self) == type(o) and self.left == o.left
  def __ne__(self, o: object) -> bool:
    return type(self) != type(o) or self.left != o.left

class Constant(UnaryOperation):
  def __init__(self, val):
    super().__init__(val)
    self.rng = (self.left, self.left+1)
  def eval(self, inputs):
    return self.left
  def __repr__(self) -> str:
    return f"{self.left}"

class Inp(UnaryOperation):
  def __init__(self, input_id):
    super().__init__(input_id)
    self.rng = (1, 10)
  def eval(self, inputs):
    return inputs[self.left]
  def __repr__(self) -> str:
    return f"input{self.left}"

class BinOperation(UnaryOperation):
  def __init__(self, left, right):
    super().__init__(left)
    self.right = right
  def __eq__(self, o: object) -> bool:
    return type(self) == type(o) and self.left == o.left and self.right == o.right
  def __ne__(self, o: object) -> bool:
    return type(self) != type(o) or self.left != o.left or self.right != o.right

class Add(BinOperation):
  def __init__(self, left, right):
    super().__init__(left, right)
    ll, lr = left.rng
    rl, rr = right.rng
    self.rng = tuple(sorted((ll+rl, lr+rr-1)))
  def eval(self, inputs):
    return self.left.eval(inputs) + self.right.eval(inputs)
  def simplify(self):
    l = self.left.simplify() 
    r = self.right.simplify()
    return Constant(l.left + r.left) if type(l) is Constant and type(r) is Constant \
      else l if r == Constant(0) \
      else r if l == Constant(0) \
      else Add(l, r)
  def __repr__(self) -> str:
    return f"({self.left} + {self.right})"

class Mul(BinOperation):
  def __init__(self, left, right):
    super().__init__(left, right)
    ll, lr = left.rng
    rl, rr = right.rng
    self.rng = tuple(sorted((ll*rl, (lr-1)*(rr-1)+1)))
  def eval(self, inputs):
    return self.left.eval(inputs) * self.right.eval(inputs)
  def simplify(self):
    l = self.left.simplify() 
    r = self.right.simplify()
    return Constant(l.left * r.left) if type(l) is Constant and type(r) is Constant \
      else l if r == Constant(1) \
      else r if l == Constant(1) \
      else Constant(0) if (r == Constant(0) or l == Constant(0)) \
      else Mul(l, r)
  def __repr__(self) -> str:
    return f"({self.left} * {self.right})"

class Div(BinOperation):
  def __init__(self, left, right):
    super().__init__(left, right)
    ll, lr = left.rng
    rl, rr = right.rng
    self.rng = tuple(sorted((int(ll/rl), int((lr-1)/(rr-1))+1)))
  def eval(self, inputs):
    return int(self.left.eval(inputs)/self.right.eval(inputs))
  def simplify(self):
    l = self.left.simplify() 
    r = self.right.simplify()
    return Constant(int(l.left / r.left)) if type(l) is Constant and type(r) is Constant \
      else l if r == Constant(1) \
      else 0 if l == Constant(0) \
      else Div(l, r)
  def __repr__(self) -> str:
    return f"({self.left} / {self.right})"

class Mod(BinOperation):
  def __init__(self, left, right):
    super().__init__(left, right)
    ll, lr = left.rng
    rl, rr = right.rng
    self.rng = (ll if ll>0 and lr<=rr else 0, lr if ll>0 and lr<=rr else rr-1)  # assume positive right operands (in fact it's always 26)
  def eval(self, inputs):
    return self.left.eval(inputs) % self.right.eval(inputs)
  def simplify(self):
    l = self.left.simplify() 
    r = self.right.simplify()
    return Constant(l.left % r.left) if type(l) is Constant and type(r) is Constant \
      else Constant(0) if r == Constant(1) \
      else Mod(l, r)
  def __repr__(self) -> str:
    return f"({self.left} % {self.right})"  

class Eql(BinOperation):
  def __init__(self, left, right):
    super().__init__(left, right)
    self.rng = (0, 2)
  def eval(self, inputs):
    return 1 if self.left.eval(inputs) == self.right.eval(inputs) else 0
  def simplify(self):
    l = self.left.simplify()
    r = self.right.simplify()
    # simplification of eql is possible for constants (direct evaluation);
    # we can check if the value ranges of left and right do not intersect,
    # in which case the result is always 0 (not equal);
    # we can simplify eql(eql(l, r), 0) to neq(l, r), 
    # there are no other combinations with eql in the input
    return (Constant(1) if l.left == r.left else Constant(0)) if type(l) is Constant and type(r) is Constant \
      else Constant(0) if l.rng[1] <= r.rng[0] or r.rng[1] <= l.rng[0] \
      else Neq(l.left, l.right) if type(l) is Eql and r == Constant(0) \
      else Neq(r.left, r.right) if type(r) is Eql and l == Constant(0) \
      else Eql(l, r)
  def __repr__(self) -> str:
    return f"(1 if {self.left} == {self.right} else 0)"  

class Neq(BinOperation):
  def __init__(self, left, right):
    super().__init__(left, right)
    self.rng = (0, 2)
  def eval(self, inputs):
    return 0 if self.left.eval(inputs) == self.right.eval(inputs) else 1
  # skip simplify() as neq is already the simplification of eql(eql(l, r), 0)
  # and there are no other combinations with eql in the input
  def __repr__(self) -> str:
    return f"(0 if {self.left} == {self.right} else 1)"  

class CachedOperation(Operation):
  def __init__(self, input_id, register, operation):
    self.input_id = input_id
    self.register = register
    self.operation = operation
    self.rng = operation.rng
    self.cache = None
  def eval(self, inputs):
    if not self.cache:
      self.cache = self.operation.eval(inputs)
    return self.cache
  def evict(self):
    self.cache = None
  def __repr__(self) -> str:
    return f"{self.register}{self.input_id}"

def parse_operand(op_str, registers):
  return registers[op_str] if op_str in "wxyz" else Constant(int(op_str))

import re
program = [tuple([g for g in re.fullmatch('(inp|add|mul|div|mod|eql) (w|x|y|z) ?(w|x|y|z|-?[0-9]+)?', line.strip()).groups()]) 
           for line in open('drive/MyDrive/AoC/2021/input24.txt')]

# 1 parse into trees of operations and simplify

registers = {
  'w': Constant(0),
  'x': Constant(0),
  'y': Constant(0),
  'z': Constant(0),
}

# each input produces a value in z before the next input is read
zs = []

input_id = -1
for cmd, left, right in program:
  if cmd == 'inp':
    input_id += 1
    registers[left] = Inp(input_id)
  elif cmd == 'add':
    registers[left] = Add(registers[left], parse_operand(right, registers)).simplify()
    # last step for each input is always 'add z, y' and this does not appear anywhere else 
    # -> wrap in a caching node (caches calls to eval() until evict() is called)
    if left == 'z' and right == 'y':  
      registers[left] = CachedOperation(input_id, left, registers[left])
      zs.append(registers[left])
  elif cmd == 'mul':
    registers[left] = Mul(registers[left], parse_operand(right, registers)).simplify()
  elif cmd == 'div':
    registers[left] = Div(registers[left], parse_operand(right, registers)).simplify()
  elif cmd == 'mod':
    registers[left] = Mod(registers[left], parse_operand(right, registers)).simplify()
  elif cmd == 'eql':
    registers[left] = Eql(registers[left], parse_operand(right, registers)).simplify()

# 2 print out the z caching nodes
for z in zs:
  print(z, z.operation) 

# prints:
"""
z0 (input0 + 8)
z1 ((z0 * 26) + (input1 + 8))
z2 ((z1 * 26) + (input2 + 12))
z3 (((z2 / 26) * ((25 * (0 if ((z2 % 26) + -8) == input3 else 1)) + 1)) + ((input3 + 10) * (0 if ((z2 % 26) + -8) == input3 else 1)))
z4 ((z3 * 26) + (input4 + 2))
z5 ((z4 * 26) + (input5 + 8))
z6 (((z5 / 26) * ((25 * (0 if ((z5 % 26) + -11) == input6 else 1)) + 1)) + ((input6 + 4) * (0 if ((z5 % 26) + -11) == input6 else 1)))
z7 ((z6 * 26) + (input7 + 9))
z8 (((z7 / 26) * ((25 * (0 if ((z7 % 26) + -3) == input8 else 1)) + 1)) + ((input8 + 10) * (0 if ((z7 % 26) + -3) == input8 else 1)))
z9 ((z8 * 26) + (input9 + 3))
z10 (((z9 / 26) * ((25 * (0 if ((z9 % 26) + -3) == input10 else 1)) + 1)) + ((input10 + 7) * (0 if ((z9 % 26) + -3) == input10 else 1)))
z11 (((z10 / 26) * ((25 * (0 if ((z10 % 26) + -1) == input11 else 1)) + 1)) + ((input11 + 7) * (0 if ((z10 % 26) + -1) == input11 else 1)))
z12 (((z11 / 26) * ((25 * (0 if ((z11 % 26) + -10) == input12 else 1)) + 1)) + ((input12 + 2) * (0 if ((z11 % 26) + -10) == input12 else 1)))
z13 (((z12 / 26) * ((25 * (0 if ((z12 % 26) + -16) == input13 else 1)) + 1)) + ((input13 + 2) * (0 if ((z12 % 26) + -16) == input13 else 1)))
"""

# from the print out of the z caching nodes, we can see that 
# - each z value depends only on the preceding z value and the current input
# - for some z values, there is a condition for the current input, related to 
#   the previous z value, which, if fulfilled, leads to smaller numbers
#   => prev_z % 26 + <constant> == current input, so we make those inputs constant
#   => the following dict has the X for each affected depth (0-based input number)
fixed_input_deltas = {3: -8, 6: -11, 8: -3, 10: -3, 11: -1, 12: -10, 13: -16}

def iterate(inputs, depth, last_0):
  if depth == 14:
    return inputs if zs[13].eval(inputs) == 0 else last_0
  l = last_0
  fixed_input = None
  if depth == 1:
    # progress
    print(inputs)
  if depth in fixed_input_deltas:
    # the z(depth-1) % 26 + <constant>
    fixed_input = zs[depth-1].eval(inputs) % 26 + fixed_input_deltas[depth]
  if not fixed_input:
    for i in range(1, 10):  # part 1, increase, last_0 is max input producing 0
    # for i in range(9, 0, -1):  # part 2, decrease, last_0 is min input producing 0
      l = iterate(inputs + [i], depth + 1, l)
      zs[depth].evict()
  elif fixed_input > 0 and fixed_input < 10:
    # input digits must be in [1..9]
    l = iterate(inputs + [fixed_input], depth + 1, l)
    zs[depth].evict()
  return l
      
"".join([str(d) for d in iterate([], 0, [])])


z0 (input0 + 8)
z1 ((z0 * 26) + (input1 + 8))
z2 ((z1 * 26) + (input2 + 12))
z3 (((z2 / 26) * ((25 * (0 if ((z2 % 26) + -8) == input3 else 1)) + 1)) + ((input3 + 10) * (0 if ((z2 % 26) + -8) == input3 else 1)))
z4 ((z3 * 26) + (input4 + 2))
z5 ((z4 * 26) + (input5 + 8))
z6 (((z5 / 26) * ((25 * (0 if ((z5 % 26) + -11) == input6 else 1)) + 1)) + ((input6 + 4) * (0 if ((z5 % 26) + -11) == input6 else 1)))
z7 ((z6 * 26) + (input7 + 9))
z8 (((z7 / 26) * ((25 * (0 if ((z7 % 26) + -3) == input8 else 1)) + 1)) + ((input8 + 10) * (0 if ((z7 % 26) + -3) == input8 else 1)))
z9 ((z8 * 26) + (input9 + 3))
z10 (((z9 / 26) * ((25 * (0 if ((z9 % 26) + -3) == input10 else 1)) + 1)) + ((input10 + 7) * (0 if ((z9 % 26) + -3) == input10 else 1)))
z11 (((z10 / 26) * ((25 * (0 if ((z10 % 26) + -1) == input11 else 1)) + 1)) + ((input11 + 7) * (0 if ((z10 % 26) + -1) == input11 else 1)))
z12 (((z11 / 26) * ((25 * (0 if ((z11 % 26) + -10) == input12 else 1)) + 1)) + ((input12 + 2) * (0 if ((z11 % 26) + -10) ==

'99598963999971'

In [ ]:
#